### Apocharmm tutorial - Basic waterbox simulation example ###

Start by importing the `apocharmm` package

In [18]:
import charmm.apocharmm as ac

In [ ]:
# setup data location
testDataPath = "../../test/data/"

: 

Import the parameters anf the system's topology (prm and psf)

In [3]:
psf = ac.CharmmPSF(testDataPath + "waterbox.psf")
prm = ac.CharmmParameters(testDataPath + "toppar_water_ions.str")

Set up the ForceManager that will drive the simulation.

In [4]:
# Create the ForceManager object using the psf and prm 
fm = ac.ForceManager(psf, prm)
# Setup box size, FFT options, cutoffs...
fm.setBoxDimensions([50., 50., 50.])
fm.setFFTGrid(48,48,48)
fm.setCtonnb(9.0)
fm.setCtofnb(10.0)
fm.setCutoff(12.0)
# Finally, initialize the ForceManager object !
fm.initialize()

The simulation state will be handled by a CharmmContext object, created from the ForceManager.

In [5]:
ctx = ac.CharmmContext(fm)

This CharmmContext handles the coordinates and velocities.

In [6]:
crd = ac.CharmmCrd(testDataPath + "waterbox.crd")
ctx.setCoordinates(crd)
ctx.assignVelocitiesAtTemperature(300.0)

We start by a short minimization of our system.

In [8]:
minimizer = ac.Minimizer()
minimizer.setSimulationContext(ctx)
minimizer.minimize(1000)

Here we will integrate using a Langevin thermostat. We create the Integrator object, then attach it to the CharmmContext. Finally, we propagate for 10 steps.

In [ ]:
integrator = ac.LangevinThermostatIntegrator(.001, 12, 300)
integrator.setSimulationContext(ctx)
integrator.propagate(10)

: 

The simulation can be monitored using various Susbcribers, responsible for creating output files at a given frequency: 
* StateSubscribers (time, energy, temperature, pressure...); 
* RestartSubscriber: outputs a Charmm-like restart file
* DcdSubscriber: saves the trajectory to a .dcd file
* ...

Once created, a Subscriber needs to be subscribed to an Integrator.

In [9]:
stateSub = ac.StateSubscriber("waterboxState.txt", 500)
dcdSub = ac.DcdSubscriber("waterboxTraj.dcd", 1000)
restartSub = ac.RestartSubscriber("waterboxRestart.res", 2000)

integrator.subscribe(stateSub)
integrator.subscriber([dcdSub, restartSub])

In [10]:
integrator.propagate(10000)

Velocities assigned at temperature 300
dof : 23496
calculated temp from ke (host) : 446.686
calculated temp from ke : 446.686
[Minimization] Didn't reach minimization tolerance. Number of interations exhausted.
Step = 0 0ms
kinetic energy = 3539.73
potential energy = -43624.8
total energy = -40085
[LangTherm]Temp : 151.624
kinetic energy = 3734.91
potential energy = -43820.1
total energy = -40085.2
[LangTherm]Temp : 159.984
kinetic energy = 3714.84
potential energy = -43800
total energy = -40085.2
[LangTherm]Temp : 159.124
kinetic energy = 3709.07
potential energy = -43794.3
total energy = -40085.2
[LangTherm]Temp : 158.877
kinetic energy = 3767.76
potential energy = -43852.9
total energy = -40085.2
[LangTherm]Temp : 161.391
kinetic energy = 3883.13
potential energy = -43968.5
total energy = -40085.4
[LangTherm]Temp : 166.333
kinetic energy = 3915.74
potential energy = -44001.1
total energy = -40085.3
[LangTherm]Temp : 167.73
kinetic energy = 3912.12
potential energy = -43997.5
total e